In [ ]:
import os
os.environ['MISTRAL_API_KEY'] = 'e7sPYrNGvXaeMFK8hgfjWb3lPSVOJrZP'

## Prepare QA

In [3]:
# ...existing code...
import json
def prepare_qa(file_path: str):
    """
    Prepares QA pairs from a JSONL file.
    """
    with open(file_path, 'r', encoding='utf-8') as file:  # Specify UTF-8 encoding
        data = [json.loads(line) for line in file]

    qa = []
    for item in data:
        qa.append({'question': item['question'], 'options': item['options'], 'answer': item['answer_idx']})
        # qa.append({'question': item['question'], 'options': {"A": item['opa'],"B": item['opb'], "C": item['opc'], "D": item['opd']}, 'answer': chr(item['cop'] + 64) })

    
    return qa

# ...existing code...

In [4]:
QA = prepare_qa(r"E:\Git_clone\RAG\qa_dataset\data_clean\questions\US\4_options\phrases_no_exclude_test.jsonl")
# QA = prepare_qa(r"C:\Users\This PC\Downloads\medmcqa\dev.json")
global count_ans
count_ans = 0

## Evaluation

In [5]:
import chatbot, importlib
importlib.reload(chatbot)
from chatbot import Chatbot
chatbot = Chatbot("mistral")

### Vector retriever

In [6]:
import vectordb
importlib.reload(vectordb)
from vectordb import create_retriever

In [7]:
vretriever = create_retriever(r"E:\workspaces\YuE\faiss_index")

### Graph retriever

In [8]:
import graphdb, importlib
importlib.reload(graphdb)

from graphdb import gretriever

In [9]:
def retrieve(query, rag_type = None, k = 5):
    if rag_type == None:
        return ""
    elif rag_type == "rag":
        contexts = vretriever.get_relevant_documents(query, k=k)
        return "\n\n".join([context.page_content for context in contexts])
    elif rag_type == "grag":
        vcontexts = vretriever.get_relevant_documents(query, k=k)
        gcontexts = gretriever(".\n".join([context.page_content for context in vcontexts]), extract_model="mistral", k=999)
        return "\n".join([context.page_content for context in vcontexts]) + "\n".join([context for context in gcontexts])

In [10]:
def process_qa(qa, rag_type = None, k = 5):
    context = retrieve(qa['question'], rag_type, k=k)
    prompt = f"""
    You are a medical expert. Answer the question by using both the provided documents and your knowledge.
    Question: {qa['question']}
    Options:
    A: {qa['options']['A']}
    B: {qa['options']['B']}
    C: {qa['options']['C']}
    D: {qa['options']['D']}
    Document: {context}
    Answer: ("[A]", "[B]", "[C]", or "[D]"). DO NOT include the context, the explaination nor the question in your answer.
    """
    response = chatbot.chat(prompt)
    # print("Prompt: ", prompt, '\n')
    if len(response) == 1:
        answer = response
    if "[A]" in response:
        answer = "A"
    elif "[B]" in response:
        answer = "B"
    elif "[C]" in response:
        answer = "C"
    elif "[D]" in response:
        answer = "D"
    else:
        answer = response[0]
    
    # print(response, " ", answer, "  ", qa['answer'])
    output_file_path = "output.txt"  # Specify the output file path
    with open(output_file_path, "a", encoding="utf-8") as file:
        file.write("1" if answer == qa['answer'] else "0")

    return answer == qa['answer']

In [11]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

def process_qa_with_retry(qa, rag_type=None, retries=8, delay=1):
    """
    Processes a single QA pair with retry logic for rate-limiting errors.

    Parameters:
        qa (dict): A single QA pair to process.
        rag_type (str): Type of retriever to use (e.g., "rag", "grag").
        retries (int): Number of retries for rate-limiting errors.
        delay (int): Delay in seconds between retries.

    Returns:
        bool: Whether the processed answer matches the expected answer.
    """
    for attempt in range(retries):
        try:
            return process_qa(qa, rag_type)
        except Exception as e:
            time.sleep(delay)
    return False  # Return False if all retries fail

def process_qa_parallel(QA, n_workers=4, rag_type=None, k=5):
    """
    Processes QA pairs in parallel and calculates accuracy.

    Parameters:
        QA (list): List of QA pairs to process.
        n_workers (int): Number of worker threads to use.
        rag_type (str): Type of retriever to use (e.g., "rag", "grag").

    Returns:
        float: Accuracy of the processed QA pairs.
    """
    count_ans = 0

    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        # Submit tasks to the executor
        futures = {executor.submit(process_qa_with_retry, qa, rag_type, k): qa for qa in QA}

        # Use tqdm for progress tracking
        for future in tqdm(as_completed(futures), total=len(QA), desc="Processing QA"):
            try:
                if future.result():  # If the result is True, increment the counter
                    count_ans += 1
            except Exception as e:
                print(f"Error processing QA: {e}")

    accuracy = count_ans / len(QA)
    return accuracy

In [ ]:
accuracy = process_qa_parallel(QA[:], n_workers=6, rag_type="rag", k=8)
print(f"Accuracy: {accuracy}")


Processing QA:  58%|█████▊    | 740/1273 [29:03<11:54,  1.34s/it]   

In [16]:
accuracy


0.7439120188531029

In [ ]:
s = "1011110011100101010100101101111110010010000110111101110101010111111111101111001110111110101011101011"
cnt = 0x
for i in s:
    if i == "1":
        cnt+=1
cnt

65